## This notebook compare the differents peaks search

In [ ]:
%matplotlib inline

import multiprocessing.pool
import timeit

import matplotlib.pyplot as plt
import torch
import tqdm

from laueimproc.io.download import get_samples
from laueimproc import Diagram

### Firstly, we have to create the diagrams from images
* To see all the ways to create a diagram, please have a look to the `init_diagram` notebook.
* Take care having enough RAM and swappiness to load all the images, otherwise it would bias the peak search time.

In [ ]:
all_files = sorted(get_samples().iterdir())  # the list of all images path
diagrams = [Diagram(f) for f in all_files]
for diagram in tqdm.tqdm(diagrams):
    diagram.image

In [ ]:
def load_images():
    with multiprocessing.pool.ThreadPool() as pool:
        pool.map(lambda d: (d.compress(), d.image), diagrams)
times = timeit.repeat(load_images, repeat=10, number=1)
print(f"on average, an image is read in {1000*min(times)/len(diagrams):.2f}ms")

### Secondly, prepare tools for spots evaluation
* Usefull to compare differents peaks search

In [ ]:
def time_peaks_search(**kwargs):
    def run():
        with multiprocessing.pool.ThreadPool() as pool:
            pool.map(lambda d: (d.find_spots(**kwargs), d.flush()), diagrams)
    times = timeit.repeat(run, repeat=5, number=1)
    print(f"on average, peaks search takes {1000*min(times)/len(diagrams):.2f}ms for one diagram")

def spots_distibution():
    nb_spots = torch.tensor([len(d) for d in diagrams], dtype=torch.float32)
    print(
        "nbr of spots: "
        f"median={round(torch.median(nb_spots).item())}, "
        f"mean={round(torch.mean(nb_spots).item())}, "
        f"min={round(torch.min(nb_spots).item())}, "
        f"max={round(torch.max(nb_spots).item())}, "
        f"std={torch.std(nb_spots).item():.1f}"
    )

def intensities_distribution():
    intensities = torch.cat([d.compute_pxl_intensities() for d in diagrams])
    print(
        "spot intensities: "
        f"median={torch.median(intensities).item():.3f}, "
        f"mean={torch.mean(intensities).item():.3f}, "
        f"min={torch.min(intensities).item():.3f}, "
        f"max={torch.max(intensities).item():.3f}, "
        f"std={torch.std(intensities).item():.3f}"
    )

### Thirdly, run the peaks search

In [ ]:
for density in (0.7, 0.6, 0.55, 0.5, 0.45, 0.4, 0.3):
    print(f"{'*'*20} DENSITY {density:.2f} {'*'*20}")
    time_peaks_search(density=density)
    # spots_distibution()
    # intensities_distribution()
    print()